In [46]:
from pathlib import Path
from ai_edge_litert.interpreter import Interpreter
import numpy as np
from PIL import Image
from typing import List, Dict
from torchvision import datasets

RESULTS_FOLDER = Path("./results")
IMAGE_FOLDER = Path("./images")
MODEL = Path("./resnet18.tflite")

CLASS_LABELS = "imagenet_classes.txt"
IMAGENETV2_FOLDER = "/scratch/msc25f15/datasets/imagenetv2-top-images/imagenetv2-top-images-format-val/"
dataset = datasets.ImageFolder(root=IMAGENETV2_FOLDER)
idx_to_class = {idx: cls for cls, idx in dataset.class_to_idx.items()} # ImageFolder creates some internal ids for the classes
filename_to_label = {path.split('/')[-1]: idx_to_class[label] for path, label in dataset.samples}
# print(filename_to_label)

def get_top5_labels(output_data: np.ndarray) -> List[str]:
    with open(CLASS_LABELS) as f:
        labels = [line.strip() for line in f]
        top5_indices = np.argsort(output_data[0])[::-1][:5]
        top5_labels = [labels[i] for i in top5_indices]
        return top5_labels

def label_id_to_name(label_id: int) -> str:
    with open(CLASS_LABELS) as f:
        labels = [line.strip() for line in f]
        return labels[label_id]

def read_cva6_result(file_path: Path) -> Dict:
    result = {}
    with open(file_path, 'r') as f:
        lines = f.readlines()
        if len(lines) < 2:
            return {}
        top5 = [line.split(":")[1].split("(")[0].strip() for line in lines[1:6]]
        result['top5'] = top5
        result['top5_confidences'] = [float(line.split(":")[2].strip(")\n")) for line in lines[1:6]]
        result["all"] = [lines[7].strip().split(",")]
        return result


class Result:
    def __init__(self, image_name: str, ground_truth: str, cva6_top5: List[str], x86_top5: List[str], cva6_confidence: List[float] = None, x86_confidence: List[float] = None):
        self.image_name = image_name
        self.ground_truth = ground_truth
        self.cva6_top5 = cva6_top5
        self.x86_top5 = x86_top5
        self.cva6_correct_t5 = ground_truth in cva6_top5
        self.x86_correct_t5 = ground_truth in x86_top5
        self.cva6_correct_t1 = ground_truth == cva6_top5[0]
        self.x86_correct_t1 = ground_truth == x86_top5[0]
        self.t1_mismatch = cva6_top5[0] != x86_top5[0]
        self.t5_mismatch = cva6_top5[0] not in x86_top5
        self.cva6_confidence = cva6_confidence
        self.x86_confidence = x86_confidence

    def print(self):
        print(f"Image: {self.image_name}")
        print(f"Ground Truth: {self.ground_truth}")
        print(f"CVA6 Correct Top-5: {self.cva6_correct_t5} | Correct Top-1: {self.cva6_correct_t1}")
        print(f"x86 Correct Top-5: {self.x86_correct_t5} | Correct Top-1: {self.x86_correct_t1}")
        print(f"CVA6 Top-5: {self.cva6_top5}")
        if self.cva6_confidence is not None:
            print(f"CVA6 Confidence: {self.cva6_confidence}")
        print(f"x86 Top-5: {self.x86_top5}")
        if self.x86_confidence is not None:
            print(f"x86 Confidence: {self.x86_confidence}")
        print("--------------------------------------------------")

    def return_conf_matrix_t1(self):
        conf_matrix = np.array([[self.cva6_correct_t1 and self.x86_correct_t1, (not self.cva6_correct_t1) and self.x86_correct_t1], [(not self.x86_correct_t1) and self.cva6_correct_t1, (not self.x86_correct_t1) and (not self.cva6_correct_t1)]])
        assert conf_matrix.sum() == 1
        return conf_matrix
    
    def return_conf_matrix_t5(self):
        conf_matrix = np.array([[self.cva6_correct_t5 and self.x86_correct_t5, (not self.cva6_correct_t5) and self.x86_correct_t5], [(not self.x86_correct_t5) and self.cva6_correct_t5, (not self.x86_correct_t5) and (not self.cva6_correct_t5)]])
        assert conf_matrix.sum() == 1
        return conf_matrix

def print_confusion_matrix_t1(results:List[Result]):
    conf_matrix = np.array([[0,0],[0,0]])
    for res in results:
        conf_matrix += np.array(res.return_conf_matrix_t1())
        
    print("Confusion Matrix Top-1")
    print(conf_matrix)
def print_confusion_matrix_t5(results:List[Result]):
    conf_matrix = np.array([[0,0],[0,0]])
    for res in results:
        conf_matrix += np.array(res.return_conf_matrix_t5())
        
    print("Confusion Matrix Top-5")
    print(conf_matrix)


results = list(RESULTS_FOLDER.glob("*.txt"))
interpreter = Interpreter(MODEL)
interpreter.allocate_tensors()
input_details = interpreter.get_input_details()
output_details = interpreter.get_output_details()
total_count = 0
top1_errors_cva6 = 0
top1_errors_x86 = 0
top5_errors_cva6 = 0
top5_errors_x86 = 0
t1_match_errors = 0
t5_match_errors = 0
res_objs = []

for result in results:
    image_path = IMAGE_FOLDER / (result.stem.replace("_results", "") + ".bmp")
    image = Image.open(image_path).convert('RGB')
    input_data = np.asarray(image, dtype=np.float32) / 255.0  # Shape: [224, 224, 3]
    input_data = np.transpose(input_data, (2, 0, 1))  # Shape: [3, 224, 224]
    input_data = np.expand_dims(input_data, axis=0)  # Shape: [1, 3, 224, 224]
    interpreter.set_tensor(input_details[0]['index'], input_data)
    interpreter.invoke()
    output_data = interpreter.get_tensor(output_details[0]['index'])
    cva6_res = read_cva6_result(result)
    x86_top5 = get_top5_labels(output_data)

    # top1_match = cva6_res['top5'][0] == x86_top5[0]
    ground_truth_label = label_id_to_name(int(filename_to_label[image_path.name.replace('.bmp','.jpeg')]))
    # if not top1_match:
    #     print(f"Mismatch detected! in file {image_path.name}")
    #     print(f"Ground Truth: {ground_truth_label}")

    #     print(f"CVA6:")
    #     print(cva6_res["top5"])
    #     print(f"Confidence: {cva6_res['top5_confidences']}")

    #     print(f"x86:")
    #     print(x86_top5)
    #     print(f"x86 confidence: {np.sort(output_data[0])[::-1][:5]}")
        # total_match_errors += 1


    res = Result(image_path.name, ground_truth_label, cva6_res['top5'], x86_top5, cva6_res['top5_confidences'], np.sort(output_data[0])[::-1][:5])
    res_objs.append(res)
    if not res.x86_correct_t5:
        top5_errors_x86 += 1
    if not res.x86_correct_t1:
        top1_errors_x86 += 1
    if not res.cva6_correct_t1:
        top1_errors_cva6 += 1
    if not res.cva6_correct_t5:
        top5_errors_cva6 += 1


    total_count += 1

print_confusion_matrix_t1(res_objs)
print_confusion_matrix_t5(res_objs)

for r in res_objs:
    if r.t1_mismatch:
        t1_match_errors += 1
        r.print()
    if r.t5_mismatch:
        t5_match_errors += 1


print(f"Total images processed: {total_count}")
print(f"CVA6 Top-1 Accuracy Rate: {(total_count-top1_errors_cva6)/total_count*100:.2f}%")
print(f"CVA6 Top-5 Accuracy Rate: {(total_count-top5_errors_cva6)/total_count*100:.2f}%")
print(f"x86 Top-1 Accuracy Rate: {(total_count-top1_errors_x86)/total_count*100:.2f}%")
print(f"x86 Top-5 Accuracy Rate: {(total_count-top5_errors_x86)/total_count*100:.2f}%")
print(f"Total Top-1 Mismatches between CVA6 and x86: {t1_match_errors}")
print(f"Total Top-5 Mismatches between CVA6 and x86: {t5_match_errors}")

Confusion Matrix Top-1
[[49  2]
 [ 1 48]]
Confusion Matrix Top-5
[[71  1]
 [ 2 26]]
Image: 7157fc4964282f306ac5342c9457df668368eabd.bmp
Ground Truth: indri
CVA6 Correct Top-5: True | Correct Top-1: False
x86 Correct Top-5: True | Correct Top-1: True
CVA6 Top-5: ['howler monkey', 'spider monkey', 'indri', 'colobus', 'langur']
CVA6 Confidence: [9.2198, 9.0343, 8.6749, 8.6708, 8.0141]
x86 Top-5: ['indri', 'spider monkey', 'howler monkey', 'colobus', 'langur']
x86 Confidence: [9.572002  9.499167  9.4655285 9.170137  8.609384 ]
--------------------------------------------------
Image: 03a82b8b2265f3625c2f9c22116cfbec09292d74.bmp
Ground Truth: entertainment center
CVA6 Correct Top-5: True | Correct Top-1: False
x86 Correct Top-5: True | Correct Top-1: True
CVA6 Top-5: ['china cabinet', 'entertainment center', 'television', 'wardrobe', 'medicine chest']
CVA6 Confidence: [11.945, 11.8862, 10.1129, 10.0865, 9.1037]
x86 Top-5: ['entertainment center', 'china cabinet', 'wardrobe', 'television', '